# COVID-19 Interactive Analysis Dashboard

## What is COVID-19?

> Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS).1
'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`

![Coronavirus particle Image](https://www.apta.com/wp-content/uploads/home-banner-1.jpg)

In [2]:
# importing libraries
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import plotly.offline as pyo
import ipywidgets as widgets



from matplotlib import style
style.use('ggplot')
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
plt.rcParams['figure.figsize'] = 25, 12

pyo.init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
df=pd.read_csv(r'https://raw.githubusercontent.com/rahulagarwal01/Covid_19_Datasets/master/Covid%20cases%20in%20India.csv')

In [3]:
# data cleaning

df.drop(['S. No.'],axis=1,inplace=True)
df['Total Cases']=df['Total Confirmed cases (Indian National)']+df['Total Confirmed cases ( Foreign National )']


# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
# country_df.head()

# COVID-19 India Stats

In [4]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

#For India
TOTAL_CASES_OVERALL=df['Total Cases'].sum()
df['Active Cases']=df['Total Cases']-df['Death']-df['Cured']
df.style.background_gradient(cmap='Reds')


,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured,Death,Total Cases,Active Cases
0,Andhra Pradesh,895,0,141,27,895,727
1,Chhattisgarh,36,0,28,0,36,8
2,Delhi,2376,1,808,50,2377,1519
3,Gujarat,2624,0,258,112,2624,2254
4,Haryana,272,14,156,3,286,127
5,Himachal Pradesh,40,0,18,1,40,21
6,Karnataka,445,0,145,17,445,283
7,Kerala,440,7,324,3,447,120
8,Madhya Pradesh,1699,0,203,83,1699,1413
9,Maharashtra,6427,3,840,283,6430,5307


# COVID-19 Total Active Cases in India

In [5]:
Total_Active_Cases= df.groupby('Name of State / UT')['Active Cases'].sum().sort_values(ascending=False).to_frame()
Total_Active_Cases.style.background_gradient(cmap='Blues')

,Active Cases
Name of State / UT,
Maharashtra,5307
Gujarat,2254
Rajasthan,1707
Delhi,1519
Madhya Pradesh,1413
Uttar Pradesh,1281
Tamil Nadu,911
Telengana,739
Andhra Pradesh,727


# COVID-19 Cases By State Chart in India

In [6]:
#to arrange use object oriented mathub

df.iplot(kind='scatter',x='Name of State / UT' ,y='Total Cases')

In [7]:
df.iplot(kind='scatter',x='Name of State / UT' ,y='Death'
         ,mode='markers+lines',title='My Graph',xTitle='Name of State / UT',yTitle='Death',colors='red',size=20)
px.scatter(df,x='Name of State / UT',y='Cured')

In [8]:
labels = ['Andhra Pradesh','Chhattisgarh','Delhi','Gujarat','Haryana','Himachal Pradesh','Karnataka','Kerala',
'Madhya Pradesh','Maharashtra','Odisha','Puducherry','Punjab','Rajasthan','Tamil Nadu','Telengana','Chandigarh',
'Jammu and Kashmir','Ladakh','Uttar Pradesh','Uttarakhand','West Bengal','Bihar','Mizoram','Goa','Manipur','Arunchal Pradesh',
'Assam','Tripura', 'Jharkhand','Meghalaya']
values = [27,0,50,112,3,1,17,3,83,283,1,0,16,27,20,24,0,5,0,24,0,20,2,0,0,0,0,1,5,3]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()


In [9]:
# displaying the total stats
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px; font-weight:bold;font-family:roboto'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;font-weight:bold;font-family:roboto'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;font-weight:bold;font-family:roboto'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

# COVID-19 Confirmed/Death/Recovered cases by countries

## Enter number of countries you want the data for

In [10]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

In [11]:
interact(show_latest_cases, n='10')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [12]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

# Slide to check for the worst hit countries

In [13]:
# # plotting the 20 worst hit countries
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

In [14]:
interact(bubble_chart, n=10)
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [15]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
        
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


# Check the details of your country or the World

* Enter the name of your country(in capitalized format(e.g. Italy)) and world for total cases

In [16]:
interact(plot_cases_of_a_country, country='World')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# 10 worst hit countries - Confirmed cases

In [17]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# 10 worst hit countries - Death cases

In [18]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# Worst hit countries - Recovering cases

In [19]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# Global spread of COVID-19

In [20]:
world_map = folium.Map(location=[11,0], tiles="cartodbdark_matter", zoom_start=2, max_zoom = 6, min_zoom = 2)
for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: -20px; background-color: black; color: white;padding-left:10px;padding-top:10px;padding-right:10px;padding-bottom:10px;font-family:roboto'>"+
                    "<h4 style='text-align:center;font-weight: bold;font-family:roboto'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;font-family:roboto'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px;font-family:roboto'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map


In [21]:
data=pd.read_csv(r'https://raw.githubusercontent.com/rahulagarwal01/Covid_19_Datasets/master/covid_19_data.csv',parse_dates=['Last Update'])
data_confirmed=pd.read_csv(r'https://raw.githubusercontent.com/rahulagarwal01/Covid_19_Datasets/master/time_series_covid_19_confirmed.csv')


In [22]:
data.rename(columns={'ObservationDate':'Date','Country/Region':'Country'},inplace=True)
data_confirmed.rename(columns={'Country/Region':'Country'},inplace=True)
confirmed=data.groupby('Date').sum()['Confirmed'].reset_index()
death=data.groupby('Date').sum()['Deaths'].reset_index()
rec=data.groupby('Date').sum()['Recovered'].reset_index()

df_latlong=pd.merge(data,data_confirmed,on=['Country','Province/State'])
fig=px.density_mapbox(df_latlong,lat="Lat",lon="Long",hover_name="Province/State",hover_data=["Confirmed","Deaths","Recovered"],animation_frame="Date",color_continuous_scale="Portland",radius=7,zoom=0,height=700)
fig.update_layout(title='Worldwide Corona Virus Cases')
fig.update_layout(mapbox_style="carto-darkmatter",mapbox_center_lon=0)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# [For Timeseries Forecasting of COVID-19 Click Here](https://project-covid19-timeseries.herokuapp.com/)

In [4]:
#import webbrowser
#button = widgets.Button(description='Click Here')
#def on_button_clicked(_):
#      webbrowser.open('https://project-covid19-timeseries.herokuapp.com/')
#button.on_click(on_button_clicked)
#widgets.VBox([button])

In [24]:
from IPython.core.display import display, HTML
display(HTML('<script src="https://www.gstatic.com/dialogflow-console/fast/messenger/bootstrap.js?v=1"></script>'
'<df-messenger intent="WELCOME" chat-title="COVID_ChatBot" agent-id="aaf92dc6-9bc5-44e8-bdce-aad65824f26f" language-code="en"></df-messenger>'))